<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">AG Dynamics of the Earth</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Jupyter notebooks</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
</tr>
</table>

# Ice-age dynamics: 6. Earth Orbit
## Simple ice-volume models
----
*Georg Kaufmann,
Geophysics Section,
Institute of Geological Sciences,
Freie Universität Berlin,
Germany*

In this notebook, we recode the examples of the **conceptional ice-volume models** from the lecture, using `python`.

We first initialize the `python` libraries.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

As a first step, we need to load the tabulated functions for the *insolation* and the *specmap archive*.
The file are called:
- `insolation.x`
- `specmap.x`
and reside in a `data` subdirectory (You can navigate to this sub-directory...)

The last two lines define the *min* and *max* values for our time scale in thousands of years [ka].

In [ ]:
#-----------------------------------------------------------------------
# read data into field
#-----------------------------------------------------------------------
infile1='data/insolation.x'
year  = np.loadtxt(fname=infile1,skiprows=1,usecols=0);year=-year
insol = np.loadtxt(fname=infile1,skiprows=1,usecols=1)
n = insol.shape[0]

infile1='data/specmap.x'
year2=np.loadtxt(fname=infile1,skiprows=1,usecols=0);year2=-year2
specmap=np.loadtxt(fname=infile1,skiprows=1,usecols=1)

tmax = 800.
tmin = 0.

print (year.min(),year.max())
print (insol.min(),insol.max())
print (specmap.min(),specmap.max())

In [ ]:
fig,axs = plt.subplots(2,1,figsize=(10.0, 6.0))

axs[0].set_xlabel('')
axs[0].set_ylabel('Insolation')
axs[0].set_title('Calder ice-volume model')
axs[0].plot(year/1000,insol,color='blue',linestyle='-',linewidth=2,label='Insolation')
axs[0].set_xlim([tmax,tmin])
axs[0].set_ylim([400,600])
axs[0].axes.get_xaxis().set_visible(False)
axs[0].spines['bottom'].set_visible(False)
axs[0].legend(loc='upper right')

axs[1].set_xlabel('Time B.P. [ky]')
axs[1].set_ylabel('SPECMAP stack')
axs[1].fill_between(year2/1000,specmap,3,color='red',linestyle='-',linewidth=2,label='Specmap')
axs[1].set_xlim([tmax,tmin])
axs[1].set_ylim([3,-3])
axs[1].spines['top'].set_visible(False)
axs[1].legend(loc='upper right')

## Calder ice-volume model

Ice volume directly coupled to insolation:
$$
\frac{\displaystyle dV}{\displaystyle dt} = -k \left( I - I_0 \right)
$$
with $I$ insolation, $I_0$ insolation threshold (ice volume increases, once
insolution is lower than threshold), $V$ ice volume, $t$ time,
and $k$ a proportionality parameter, depending on *ablation* or
*accumulation*:
    
- $k=k_M$ für $I>I_0$, melting
- $k=k_A$ für $I<I_0$, accumulation

In [ ]:
#-----------------------------------------------------------------------
# define function to plot interactively
#-----------------------------------------------------------------------
def plot_calder(I0,km,ka):
    """
    Calder ice-volume model
    """
    dtime  = 1000.
    volume = np.zeros(n)
    for i in range(1,n):
        k = ka
        if (insol[i] > I0):
            k = km
        volume[i] = volume[i-1] - k*(insol[i]-I0)*dtime
        if (volume[i] < 0):
            volume[i] = 0
    volume = volume-volume.min()
    volume = volume / volume.max()
        
    # plot time series
    plt.figure(figsize=(10.0, 6.0))
    plot1=plt.subplot2grid((3,2), (0,0), colspan=2,rowspan=1)
    plot2=plt.subplot2grid((3,2), (1,0), colspan=2,rowspan=1)
    plot3=plt.subplot2grid((3,2), (2,0), colspan=2,rowspan=1)
    plot1.set_xlabel('')
    plot1.set_ylabel('Insolation')
    plot1.set_title('Calder ice-volume model')
    plot1.plot(year/1000,insol,color='blue',linestyle='-',linewidth=2,label='Insolation')
    plot1.set_xlim([tmax,tmin])
    plot1.set_ylim([400,600])
    plot1.axes.get_xaxis().set_visible(False)
    plot1.spines['bottom'].set_visible(False)
    #plot1.legend(loc='upper right')
    
    plot2.set_xlabel('Time B.P. [ky]')
    plot2.set_ylabel('Volume')
    plot2.fill_between(year/1000,0,volume,color='black',linestyle='-',linewidth=2,label='Specmap')
    plot2.set_xlim([tmax,tmin])
    plot2.set_ylim([1.0,0])
    plot2.set_yticks([1])
    plot2.axes.get_xaxis().set_visible(False)
    plot2.spines['top'].set_visible(False)
    plot2.spines['bottom'].set_visible(False)
    #plot2.legend(loc='upper right')
    
    plot3.set_xlabel('Time B.P. [ky]')
    plot3.set_ylabel('SPECMAP stack')
    plot3.fill_between(year2/1000,specmap,3,color='red',linestyle='-',linewidth=2,label='Specmap')
    plot3.set_xlim([tmax,tmin])
    plot3.set_ylim([3,-3])
    plot3.spines['top'].set_visible(False)
    #plot3.legend(loc='upper right')

km     = 0.001
ka     = 0.0002 #0.2*km
I0     = 502.
#plot_calder(I0,km,ka)
    
# call interactive module
w = dict(
km=widgets.FloatSlider(min=0.001,max=0.005,step=0.001,value=0.001,description='km'),
ka=widgets.FloatSlider(min=0.0002,max=0.0005,step=0.0001,value=0.0002,description='ka'),
I0=widgets.IntSlider(min=460,max=515,step=2,value=502,description='I0'))

output = widgets.interactive_output(plot_calder, w)
box = widgets.HBox([widgets.VBox([*w.values()]), output])
display(box)

## Imbrie & Imbrie ice-volume model

Ice volume increases with delay:
$$
\frac{\displaystyle dV}{\displaystyle dt} = \frac{\displaystyle I-V}{\displaystyle \tau}
$$
with $I$ insolation, $V$ ice volume, $t$ time,
and $\tau$ a retardation time, depending on *ablation* or
*accumulation*:
    
- $\tau=\tau_M$ für $V>I$, melting
- $\tau=\tau_A$ für $V<I$, accumulation

In [ ]:
#-----------------------------------------------------------------------
# define function to plot interactively
#-----------------------------------------------------------------------
def plot_imbrie(taum,taua):
    """
    Imbrie & Imbrie ice-volume model
    """
    dtime  = 1000.
    volume = np.zeros(n)
    volume[0] = insol[0]
    for i in range(1,n):
        tau = taua
        if (volume[i-1] > insol[i]):
            tau = taum
        volume[i] = volume[i-1] + (insol[i]-volume[i-1])/tau*dtime
        if (volume[i] < 0):
            volume[i] = 0
    #print (volume.min(),volume.max())
    volume = volume-volume.max()
    volume = volume / volume.min()
    #print (volume.min(),volume.max())
        
    # plot time series
    plt.figure(figsize=(10.0, 6.0))
    plot1=plt.subplot2grid((3,2), (0,0), colspan=2,rowspan=1)
    plot2=plt.subplot2grid((3,2), (1,0), colspan=2,rowspan=1)
    plot3=plt.subplot2grid((3,2), (2,0), colspan=2,rowspan=1)
    plot1.set_xlabel('')
    plot1.set_ylabel('Insolation')
    plot1.set_title('Imbrie&Imbrie ice-volume model')
    plot1.plot(year/1000,insol,color='blue',linestyle='-',linewidth=2,label='Insolation')
    plot1.set_xlim([tmax,tmin])
    plot1.set_ylim([400,600])
    plot1.axes.get_xaxis().set_visible(False)
    plot1.spines['bottom'].set_visible(False)
    #plot1.legend(loc='upper right')
    
    plot2.set_xlabel('Time B.P. [ky]')
    plot2.set_ylabel('Volume')
    plot2.fill_between(year/1000,0.,volume,color='black',linestyle='-',linewidth=2,label='Specmap')
    plot2.set_xlim([tmax,tmin])
    plot2.set_ylim([1.0,0])
    plot2.set_yticks([1])
    plot2.axes.get_xaxis().set_visible(False)
    plot2.spines['top'].set_visible(False)
    plot2.spines['bottom'].set_visible(False)
    #plot2.legend(loc='upper right')
    
    plot3.set_xlabel('Time B.P. [ky]')
    plot3.set_ylabel('SPECMAP stack')
    plot3.fill_between(year2/1000,specmap,3,color='red',linestyle='-',linewidth=2,label='Specmap')
    plot3.set_xlim([tmax,tmin])
    plot3.set_ylim([3,-3])
    plot3.spines['top'].set_visible(False)
    #plot3.legend(loc='upper right')
    
# call interactive module
w = dict(
taum=widgets.IntSlider(min=10000,max=50000,step=1000,value=42000,description='taum'),
taua=widgets.IntSlider(min=10000,max=50000,step=1000,value=10000,description='taua'))

output = widgets.interactive_output(plot_imbrie, w)
box = widgets.HBox([widgets.VBox([*w.values()]), output])
display(box)

## Paillard ice-volume model

The Paillard ice-volume model has three stages:
- $i$: interglacial
- $g$: mild glacial
- $G$: full glacial

The ice volume changes according to:
$$
\frac{\displaystyle dV}{\displaystyle dt} = \frac{\displaystyle V_{i,g,G}-V}{\displaystyle \tau_{i,g,G}}
                                          - \frac{\displaystyle F}{\displaystyle \tau_f}
$$
Here, $F$ is the normalized insolation, 
$\tau_f$ a scaling factor for $F$, $F(I)$, calculated an smoothed trunction:
$$
F(i) = 0.5* \left( I + \sqrt{4a^2 + I^2} \right)
$$
with $a$ the ratio for truncation ($=1$, fill truncation, $a=\infty$, no truncation).
Note that $I$ has been normalised before the calculation, and $F$ after calculation, according to:
$$
\begin{array}{rcl}
I &=& \frac{\displaystyle I-mean(I)}{\displaystyle std(I)} \\
F &=& \frac{\displaystyle I-mean(F)}{\displaystyle std(F)}
\end{array}
$$
with $mean$ the mean, and $std$ the standard deviation.

In [ ]:
#-----------------------------------------------------------------------
# define function to plot interactively
#-----------------------------------------------------------------------
def plot_paillard(a,tau_i,tau_g,tau_G):
    """
    Paillard ice-volume model
    """
    tau_f  = 25.
    ref_i  = 0.
    ref_g  = 1.0
    ref_G = 1.001
    i1     = 0.
    i2     = -0.75
    i3     = 1.
    # normalise insolation data
    insol1 = (insol-insol.mean()) / insol.std()
    insol2 = 0.5*(insol1 + np.sqrt(4.0+a**2 + insol1**2))
    insol3 = (insol2-insol2.mean()) / insol2.std()
    #print("%s %10.2f %10.2f %10.2f %10.2f" % ('insol3: ',insol3.min(),insol3.max(),insol3.mean(),insol3.std()))
    # calculate model
    dtime  = 1000.
    volume = np.zeros(n)
    volume[0] = 0.75
    ref = ref_g
    tau = tau_g
    for i in range(1,n-1):
        volume[i] = volume[i-1] + (ref - volume[i-1]) / tau - insol3[i+1] / tau_f 
        if ((ref == ref_G) and (insol3[i] > i1)):
            ref = ref_i
        elif ((ref == ref_i) and (insol3[i] < i2)):
            ref = ref_g
        elif ((ref == ref_g) and (volume[i] > i3)):
            ref = ref_G
        else:
            ref = ref
        if (ref == ref_G):
            tau = tau_G
        if (ref == ref_g):
            tau = tau_g
        if (ref == ref_i):
            tau = tau_i
 
        
    # plot time series
    plt.figure(figsize=(10.0, 6.0))
    plot1=plt.subplot2grid((3,2), (0,0), colspan=2,rowspan=1)
    plot2=plt.subplot2grid((3,2), (1,0), colspan=2,rowspan=1)
    plot3=plt.subplot2grid((3,2), (2,0), colspan=2,rowspan=1)
    plot1.set_xlabel('')
    plot1.set_ylabel('Insolation')
    plot1.set_title('Imbrie&Imbrie ice-volume model')
    plot1.plot(year/1000,insol3,color='blue',linestyle='-',linewidth=1,label='Norm. Insolation')
    plot1.set_xlim([tmax,tmin])
    plot1.set_ylim([-2,3])
    plot1.axes.get_xaxis().set_visible(False)
    plot1.spines['bottom'].set_visible(False)
    plot1.legend(loc='upper right')
    
    plot2.set_xlabel('Time B.P. [ky]')
    plot2.set_ylabel('Volume')
    plot2.fill_between(year/1000,0.,volume,color='black',linestyle='-',linewidth=2,label='Specmap')
    plot2.set_xlim([tmax,tmin])
    plot2.set_ylim([1.2,0])
    plot2.set_yticks([1])
    plot2.axes.get_xaxis().set_visible(False)
    plot2.spines['top'].set_visible(False)
    plot2.spines['bottom'].set_visible(False)
    #plot2.legend(loc='upper right')
    
    plot3.set_xlabel('Time B.P. [ky]')
    plot3.set_ylabel('SPECMAP stack')
    plot3.fill_between(year2/1000,specmap,3,color='red',linestyle='-',linewidth=2,label='Specmap')
    plot3.set_xlim([tmax,tmin])
    plot3.set_ylim([3,-3])
    plot3.spines['top'].set_visible(False)
    #plot3.legend(loc='upper right')
    
#call interactive module
w = dict(
a=widgets.FloatSlider(min=0,max=50,step=0.5,value=1,description='a'),
tau_i=widgets.IntSlider(min=1,max=50,step=1,value=10,description='tau_i'),
tau_g=widgets.IntSlider(min=30,max=70,step=1,value=50,description='tau_g'),
tau_G=widgets.IntSlider(min=1,max=50,step=1,value=10,description='tau_G'))

output = widgets.interactive_output(plot_paillard, w)
box = widgets.HBox([widgets.VBox([*w.values()]), output])
display(box)

The model is fairly robust:

- Any value of $\tau_G$ will do ($\tau_G \in [0,\infty]$).
- $\tau_i \in [0,15]$ kyr.
- $\tau_g \in [47,57]$ kyr.
- $a \in [0.54,1.66]$.